In [1]:
from glob import glob
import pandas as pd

In [2]:
def get_dataset(file_names, auth):
    dataset = pd.read_csv(file_names[0], sep=" ").iloc[:,0:4]
    col_names = dataset.columns
    dataset = dataset.rename(columns={col_names[0]:'sample', col_names[1]: 'degree_x',
                           col_names[2]:'degree_y', col_names[3]: 'is_valid'})
    i = 0
    prob = []
    for file in file_names[1:]:
        i = i + 1
        sample = pd.read_csv(file, sep = " ").iloc[:,0:4]
        col_names = sample.columns
        sample = sample.rename(columns={col_names[0]:'sample', col_names[1]: 'degree_x',
                           col_names[2]:'degree_y', col_names[3]: 'is_valid'})
        dataset = pd.concat([dataset, sample])
        
        if sample.shape[1] != 4:
            prob = prob + [i]
        
    dataset['y'] = auth
    
    return dataset, prob

In [3]:
file_names = glob(r"D:/ETPAD.v2/LIVE EYE MOVEMENTS/*.txt",  recursive = True)
data1,problem = get_dataset(file_names, 0)

In [6]:
data1.shape, problem

((6000000, 5), [])

data1.to_csv('data1.csv')

In [7]:
file_names = glob(r"D:/ETPAD.v2/SAS_I EYE MOVEMENTS/*.txt",  recursive = True)
data2,problem2 = get_dataset(file_names, 1)

In [8]:
data2.shape, problem2

((6000000, 5), [])

In [9]:
file_names = glob(r"D:/ETPAD.v2/SAS_II EYE MOVEMENTS/*.txt",  recursive = True)
data3,problem3 = get_dataset(file_names, 2)

In [10]:
data3.shape,problem3

((6000000, 5), [])

data2.to_csv('data2.csv')
data3.to_csv('data3.csv')

In [11]:
dataset = pd.concat([data1, data2, data3])

In [12]:
dataset.to_csv('dataset.csv')
dataset = pd.read_csv(r"dataset.csv")

In [13]:
def generate_numpy_arrays(dataset):
    prev_i = 0
    i = 0
    first = 1
    count_1 = 0
    count_2 = 0
    c1 = []
    c2= []
    x = []
    y = []
    x_train = []
    y_train = []
    k = 0
    while prev_i < 18000000:
        k+=1
        while i+1500 < k*15000:        
            window_x = dataset.iloc[i:i+1500, 1].to_list()
            window_y = dataset.iloc[i:i+1500, 2].to_list()
            x.append([window_x, window_y])
            y.append(dataset.iloc[i, 4])
            first = 0
            i = i + 125
            c1 = c1+ [len(x)]     

        prev_i = prev_i + 15000

    return np.array(x), np.array(y)

In [15]:
import numpy as np

In [16]:
X_train, y_train = generate_numpy_arrays(dataset)
np.save('train_eptad_data.npy', X_train)
np.save('train_eptad_labels.npy', y_train)